In [1]:
import cv2
import numpy as np

img = cv2.imread('rice.tif', cv2.IMREAD_UNCHANGED)
def show_image(image, window_name='Image'):
    # Display the image in a window
    cv2.imshow(window_name, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [2]:
# m = dst_image -dst3
# m1 = dst_image -dst2
# dst_image[1,src_y1==src_y2]

In [3]:
# realize bilinear interpolation function
import cv2
import numpy as np
src_image = cv2.imread('rice.tif', cv2.IMREAD_UNCHANGED)
def bilinear_wufengyang(src_image, dst_dim):
    dst_image = np.zeros(dst_dim, dtype=src_image.dtype)
    src_width, src_height = src_image.shape[1], src_image.shape[0]
    dst_width, dst_height = dst_image.shape[1], dst_image.shape[0]
    src_y = np.linspace(0, src_width-1, dst_width)
    src_x = np.linspace(0, src_height-1, dst_height)
    src_x1 = np.floor(src_x).astype(np.int32)
    src_x2 = np.ceil(src_x).astype(np.int32)
    src_y1 = np.floor(src_y).astype(np.int32)
    src_y2 = np.ceil(src_y).astype(np.int32)
    for i in range(dst_height):
        Q11 = src_image[src_x1[i], src_y1]
        Q21 = src_image[src_x2[i], src_y1]
        Q12 = src_image[src_x1[i], src_y2]
        Q22 = src_image[src_x2[i], src_y2]
        if src_x1[i] != src_x2[i]:
            w11 = (src_x2[i] - src_x[i]) * (src_y2 - src_y)
            w21 = (src_x[i] - src_x1[i]) * (src_y2 - src_y)
            w12 = (src_x2[i] - src_x[i]) * (src_y - src_y1)
            w22 = (src_x[i] - src_x1[i]) * (src_y - src_y1)
            dst_image[i,:] = w11 * Q11 + w21 * Q21 + w12 * Q12 + w22 * Q22+(src_y1==src_y2)*((src_x2[i] - src_x[i])*Q11+(src_x[i] - src_x1[i])*Q21)
            # dst_image[i,src_y1==src_y2] = Q11[src_y1==src_y2]
        else:
            w11 = (src_y2 - src_y)
            w21 = (src_y - src_y1)
            dst_image[i,:] = w11 * Q11 + w21 * Q12 +(src_y1==src_y2)*Q11
            
    return dst_image
# cv2.imshow('src_image', src_image)
dst_dim = (round(src_image.shape[0]*1.4), round(src_image.shape[1]*1.4))
dst_image = bilinear_interpolation(src_image, dst_dim)
dst2 = bilinear_wufengyang(src_image, dst_dim)
dst3 = cv2.resize(src_image, dst_dim, interpolation=cv2.INTER_LINEAR)
# cv2.imshow('dst_image', dst_image)
# cv2.imshow('dst2', dst2)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# dst_image[i,:] = w11 * Q11 + w21 * Q21 + w12 * Q12 + w22 * Q22

NameError: name 'bilinear_interpolation' is not defined

In [11]:
import timeit
import cv2

# Load an example image
 # Make sure this path is correct


# Function to run timeit on
def time_bilinear_interpolation():
    bilinear_interpolation(src_image, dst_dim)


# Timing the function
number_of_runs = 100
execution_time = timeit.timeit(time_bilinear_interpolation, number=number_of_runs)

# Calculate seconds per run
seconds_per_run = execution_time / number_of_runs

print(f"Total execution time: {execution_time:.6f} seconds for {number_of_runs} runs")
print(f"Average time per run: {seconds_per_run:.6f} seconds")


Total execution time: 1.944176 seconds for 100 runs
Average time per run: 0.019442 seconds


In [4]:
def bilinear_wufengyang(input_file, dim):
    h,w = input_file.shape[:2]
    nh,nw = dim
    nh = round(nh)
    nw = round(nw)
    outputfile = np.zeros((nh,nw),dtype=input_file.dtype)
    
    x_ratio=w/nw
    y_ratio=h/nh

    for i in range(nh):
        for j in range(nw):
            x=j*x_ratio
            y=i*y_ratio
            
            x1=int(x)
            y1=int(y)
            x2=min(x1+1,w-1)
            y2=min(y1+1,h-1)
            
            if x1==x2 and y1==y2:
                outputfile[i,j]=input_file[y1,x1]
            elif x1==x2:
                outputfile[i,j]=(input_file[y1,x1]*(y2-y)+input_file[y2,x1]*(y-y1))/(y2-y1)
            elif y1==y2:
                outputfile[i,j]=(input_file[y1,x1]*(x2-x)+input_file[y1,x2]*(x-x1))/(x2-x1)
            else:
                f11=input_file[y1,x1]
                f12=input_file[y1,x2]
                f21=input_file[y2,x1]
                f22=input_file[y2,x2]
                
                outputfile[i,j] = np.array([y2-y,y-y1])@np.array([[f11,f12],[f21,f22]])@np.array([x2-x,x-x1]).T
    return outputfile
        

In [4]:
def nearest_wufengyang(input_file,dim):
    import numpy as np
    h,w = input_file.shape[:2]
    nh,nw = dim
    nh = round(nh)
    nw = round(nw)
    outputfile = np.zeros((nh,nw),dtype=input_file.dtype)
    x_ratio = (w-1)/(nw-1)
    y_ratio = (h-1)/(nh-1)
    for i in range(nh):
        for j in range(nw):
            x = round(j*x_ratio)
            y = round(i*y_ratio)
            outputfile[i,j] = input_file[y,x]
    return outputfile

In [5]:
from scipy.interpolate import interp2d
def cubic_wufengyang(input_file,dim):
    import numpy as np
    h,w = input_file.shape[:2]
    nh,nw=dim
    nh = round(nh)
    nw = round(nw)
    outputfile = np.zeros((nh,nw),dtype=input_file.dtype)
    x = np.linspace(0,w-1,w)
    y = np.linspace(0,h-1,h)
    interpolate_function = interp2d(x,y,input_file,kind='cubic')
    xnew = np.linspace(0,w-1,nw)
    ynew = np.linspace(0,h-1,nh)
    outputfile = interpolate_function(xnew,ynew)
    return outputfile


In [7]:

import numpy as np

# Define dimensions for the images to be generated
dim1 = (int(256*1.4),int(256*1.4))
dim2 = (int(256*0.6),int(256*0.6))

# Generate images using different interpolation methods
big_nearest = nearest_wufengyang(img,dim1)
small_nearest = nearest_wufengyang(img,dim2)
big_bi = bilinear_wufengyang(img,dim1)
small_bi = bilinear_wufengyang(img,dim2)
big_cubic = cubic_wufengyang(img,dim1)
small_cubic = cubic_wufengyang(img,dim2)

# Compile all generated images into a list
images = [big_nearest,big_bi,big_cubic,small_nearest,small_bi,small_cubic]

# Define titles for the images
titles = ['Nearest Neighbor','Bilinear Interpolation','Cubic Interpolation','Nearest Neighbor (Smaller Image)','Bilinear Interpolation (Smaller Image)','Cubic Interpolation (Smaller Image)']

# Set padding
padding = 10  # You can adjust this value

# Create a blank image with enough height to hold all images and padding
total_height = images[0].shape[0]*3 + 2 * padding
concatenated_image = np.zeros((total_height, images[0].shape[1]), dtype=np.uint8)
concatenated_image[:images[0].shape[0], :images[0].shape[1]] = images[0]
concatenated_image[images[0].shape[0]+padding:2*images[0].shape[0]+padding, :images[0].shape[1]] = images[1]
concatenated_image[2*images[0].shape[0]+2*padding:3*images[0].shape[0]+2*padding, :images[0].shape[1]] = images[2]
cv2.imwrite('concatenated_image_big.jpg', concatenated_image)
#展示图片
cv2.imshow('concatenated_image_big', concatenated_image)



# Create a blank image with enough width to hold smaller images and padding
total_width = images[3].shape[1]*3 + 2 * padding
concatenated_image = np.zeros((images[3].shape[0], total_width), dtype=np.uint8)
concatenated_image[:, :images[3].shape[1]] = images[3]
concatenated_image[:, images[3].shape[1]+padding:2*images[3].shape[1]+padding] = images[4]
concatenated_image[:, 2*images[3].shape[1]+2*padding:3*images[3].shape[1]+2*padding] = images[5]

# Save the concatenated image
cv2.imwrite('concatenated_image_small.jpg', concatenated_image)
#展示图片
cv2.imshow('concatenated_image_small', concatenated_image)
cv2.waitKey(0)

/tmp/ipykernel_21884/2209132475.py:11: DeprecationWarning: `interp2d` is deprecated!
`interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.13.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids use `RegularGridInterpolator` instead.
For scattered data, prefer `LinearNDInterpolator` or
`CloughTocher2DInterpolator`.

For more details see
`https://scipy.github.io/devdocs/notebooks/interp_transition_guide.html`

  interpolate_function = interp2d(x,y,input_file,kind='cubic')
/tmp/ipykernel_21884/2209132475.py:14: DeprecationWarning:         `interp2d` is deprecated!
        `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.13.0.

        For legacy code, nearly bug-for-bug compatible replacements are
        `RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
        scattered 2D data.

        In new cod

113

: 